In [191]:
import urllib3
urllib3.disable_warnings()
import json
import datetime
import csv
import time
import pandas as pd

In [192]:
app_id = "133254600775938"

app_secret = "c24db12ccb6a583d8e212600c1bed43e" # DO NOT SHARE WITH ANYONE!

access_token = app_id + "|" + app_secret

#page_id = 'nytimes'
#page_id = 'www.salesintacc.com.ar'
#page_id = 'upsoclsabores'
page_id = 'cucinareok'

In [193]:
def request_until_succeed(url):
    http = urllib3.PoolManager()
    req = http.request('GET', url)
    success = False
    while success is False:
        try:
            response = http.urlopen('GET',url)
            if response.status == 200:
                success = True
        except:
            time.sleep(5)
            
            print ("Error for URL %s: %s" % (url, datetime.datetime.now()))

    return req

In [194]:
def getFacebookPageFeedData(page_id, access_token, num_statuses):
    
    # construct the URL string
    base = "https://graph.facebook.com/v2.11" #cambié v2.4 a v2.11, funcionaba igual con v2.4
    node = "/" + page_id + "/feed"
    parameters = "/?fields=message,link,created_time,type,name,id,comments,from,attachment,shares&limit=%s&access_token=%s" % (num_statuses, access_token)
    url = base + node + parameters
    
    # Esto está cambiado respecto al ejemplo
    http = urllib3.PoolManager()
    req = http.request('GET', url)
    data = json.loads(request_until_succeed(url).data.decode('latin-1'))
    return data
    

In [195]:
#Estoy levantando 100 posts
salida_all = getFacebookPageFeedData(page_id, access_token, 100)

columns = ['post_from', 'post_id', 'post_name', 'post_type', 'post_message', 'post_link', 'post_shares', 'created_time']

df_posts = pd.DataFrame(columns=columns)

In [196]:
def scrapPosts(post):
    post_from = '' if 'from' not in post.keys() else post['from']['id']
    post_id = post['id']
    post_name = '' if 'name' not in post.keys() else post['name']
    post_type = '' if 'type' not in post.keys() else post['type']
    post_message = '' if 'message' not in post.keys() else post['message']
    post_link = '' if 'link' not in post.keys() else post['link']
    post_shares = '' if 'shares' not in post.keys() else post['shares']['count']
    created_time = datetime.datetime.strptime(post['created_time'],'%Y-%m-%dT%H:%M:%S+0000')
    created_time = created_time + datetime.timedelta(hours=-5) # EST
    created_time = created_time.strftime('%Y-%m-%d %H:%M:%S')    

    return(post_from, post_id, post_name, post_type, post_message, post_link, post_shares, created_time)


In [197]:
n_posts = len(salida_all['data'])

for i in range(n_posts):
    df = pd.DataFrame(list(scrapPosts(salida_all["data"][i])),index = ['post_from', 'post_id', 'post_name', 'post_type', 'post_message', 'post_link', 'post_shares', 'created_time']).transpose()
    df_posts = df_posts.append(df, ignore_index=True)    


In [198]:
df_posts.to_csv("df_posts.csv")

In [199]:
def scrapComments(pos, com):

    post_id = salida_all["data"][pos]["id"]
    comm_id = salida_all["data"][pos]["comments"]["data"][com]["id"]
    comm_msg = salida_all["data"][pos]["comments"]["data"][com]["message"]
    comm_date = datetime.datetime.strptime(salida_all["data"][pos]["comments"]["data"][com]["created_time"],'%Y-%m-%dT%H:%M:%S+0000')
    comm_date = comm_date + datetime.timedelta(hours=-5) # EST
    comm_date = comm_date.strftime('%Y-%m-%d %H:%M:%S') # best time format for spreadsheet programs
    
    return(post_id, comm_id, comm_msg, comm_date)    


In [200]:
df_comms = pd.DataFrame(columns=['post_id','comm_id','comm_msg','comm_date'])

In [204]:
#n_posts = len(salida_all['data'])

for i in range(n_posts):
    if len(salida_all["data"][i]) == 8:
        continue
    ncomm = len(salida_all["data"][i]["comments"]["data"])
    for c in range(ncomm):
        df = pd.DataFrame(list(scrapComments(i,c)), index=['post_id','comm_id','comm_msg','comm_date']).transpose()
        df_comms = df_comms.append(df, ignore_index=True)
        
        

In [202]:
df_comms.to_csv("df_comms.csv")

In [205]:
df_comms

,post_id,comm_id,comm_msg,comm_date
0,262755230730537_543586662647391,543586662647391_543713655968025,Ohhh mira Agos Flores me parece que tenes que ...,2017-11-16 23:22:30
1,262755230730537_543586662647391,543586662647391_543712319301492,,2017-11-16 23:16:12
2,262755230730537_543586662647391,543586662647391_543713612634696,Mmm que bueno!!,2017-11-16 23:22:19
3,262755230730537_543586662647391,543586662647391_543712279301496,Se come caliente?,2017-11-16 23:16:04
4,262755230730537_543586662647391,543586662647391_543712785968112,Genia,2017-11-16 23:18:00
5,262755230730537_543586662647391,543586662647391_543712715968119,Geniaaa,2017-11-16 23:17:50
6,262755230730537_543586662647391,543586662647391_543712369301487,Mucho dulce !!!,2017-11-16 23:16:20
7,262755230730537_543586662647391,543586662647391_543712725968118,Que rico lo voy a realizar gracias,2017-11-16 23:17:51
8,262755230730537_543586662647391,543586662647391_543713299301394,Receta x fa,2017-11-16 23:20:21
9,262755230730537_543586662647391,543586662647391_543712025968188,Lorena,2017-11-16 23:14:54
